In [2]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from hand_tracker import HandTracker

In [3]:
def sign_reco(hand_keypoint_detection, sign_classifier, labels, img):    
    #        8   12  16  20
    #        |   |   |   |
    #        7   11  15  19
    #    4   |   |   |   |
    #    |   6   10  14  18
    #    3   |   |   |   |
    #    |   5---9---13--17
    #    2    \         /
    #     \    \       /
    #      1    \     /
    #       \    \   /
    #        ------0-
    connections = [(0, 1), (1, 2), (2, 3), (3, 4),
                   (5, 6), (6, 7), (7, 8),
                   (9, 10), (10, 11), (11, 12),
                   (13, 14), (14, 15), (15, 16),
                   (17, 18), (18, 19), (19, 20),
                   (0, 5), (5, 9), (9, 13), (13, 17), (0, 17)]
                 
    detector = HandTracker("models/palm_detection_without_custom_op.tflite",
                           "models/hand_landmark.tflite",
                           "models/anchors.csv",
                           box_shift=0.2,
                           box_enlarge=1.3)
    
    points, _ = detector(img)
    if points is not None:
        for point in points:
            x, y = point
            cv2.circle(img, (int(x), int(y)), 3, (0, 255, 0), 2)
        for connection in connections:
            x0, y0 = points[connection[0]]
            x1, y1 = points[connection[1]]
            cv2.line(img, (int(x0), int(y0)), (int(x1), int(y1)), (255, 0, 0), 1)
            
        sign_coords = points.flatten() / float(img.shape[0]) - 0.5
        sign_class = sign_classifier.predict(np.expand_dims(sign_coords, axis=0))
        sign_text = labels[sign_class.argmax()]
        wrist_x = int(points[0][0])
        wrist_y = int(points[0][1])
        cv2.putText(img, sign_text, (wrist_x-20,wrist_y+10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    else:
        return False, img
    
        
    return True, img

In [5]:
# Create sign recognition model
SIGNS = ['', 'Metal', 'Peace', 'Stop', 'OK']
sign_classifier = load_model('models/model.h5') 

In [6]:
# Starting video streaming
cv2.namedWindow('Subject zero')
camera = cv2.VideoCapture(0)

while True:
    frame = camera.read()[1]
    
    succes_flag, frame_det = sign_reco('todo:hand_keypoint_detection model', sign_classifier, SIGNS, frame)
    
    # Show frame
    if succes_flag:
        cv2.imshow('Subject zero', frame_det)
    else:
        cv2.imshow('Subject zero', frame)

    # When the video window is the active window you can close it by pressing q key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()

C:\Users\admin\Desktop\handsign projekt\hand_tracker.py:95: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x) )
